In [16]:
import csv
from dotenv import load_dotenv
import os
import requests
load_dotenv()

# get ESTA_API_KEY
api_key = os.environ.get('ESTA_API_KEY')

# set url
url = 'https://d09uj3e9kh.execute-api.us-east-1.amazonaws.com/dev'

In [17]:
answers = []
with open('data/questions.csv', 'r', encoding='utf-8') as file:
  reader = csv.reader(file)
  rows = list(reader)
  for row in rows[0:2]:
    print(row)
    req = requests.get(url, 
                       params={'query': row[0], 'sessionId': '123', 'collectionName':'levio-hr'}, 
                       headers={'x-api-key': api_key},)
    answers.append(req.json())

["Qui est le président de l'entreprise Levio?"]
['Quelle est la principale richesse de Levio?']


In [18]:
answers

[{'message': 'Missing Authentication Token'},
 {'message': 'Missing Authentication Token'}]